# Serve Docs
This is a utlity to serve the docs locally.

1. Run a CLUE instance locally
2. Configure and run the cell below.
3. A list of links to to the documents as raw json and in document viewer will be visible at this url {{ insert url here }}




In [2]:
import os

starting_dir = os.getcwd()
if starting_dir.endswith('documents'):
    os.chdir('..')

print(os.getcwd())

AttributeError: 'builtin_function_or_method' object has no attribute 'endswith'